# **Week #8 - Introduction to ML & ML Pipeline**

Data Pipeline Course - Sekolah Engineer - Pacmann Academy

**Outline**

1. Review:
    - Introduction to Machine Learning
    - Introduction to Machine Learning Pipeline
2. Case Study: Bank Marketing

# <font color='blue'>Review
---

## Introduction to Machine Learning
---

### What is Machine Learning
---

- How does Machine Learning work to predict a House Pricing?
- Machine Learning models will take a lot of examples from data
    - ML Model gets input data
    - ML Model will train or learn the pattern of data
    - ML Model predicts based on the information they learned previously	

- Say, the ML model wants to predict a house that has a price of Rp 750 juta
- To predict that house price, the ML Model must learn the pattern of the data with a house price of Rp 750 juta. Like LT, LB, Total floors, etc
- There are also types of Machine Learning:
    - Supervised Learning: the data already has a label. Based on that, the ML Model will predict the correct label
    - Unsupervised Learning: the data doesn’t have any label. So, the ML Model will find the patterns or groups of the data

### Data Engineering and Machine Learning
---

- Why do Data engineers learn Machine Learning? Isn’t it our job to build a Data Pipeline?
- There’s a possibility that the output of our Data Pipeline will be used by Data Scientists to build an ML Model
- So, we must also know how’s their workflow and what kind of product they built

<center>
    <img src="https://sekolahdata-assets.s3.ap-southeast-1.amazonaws.com/notebook-images/mde-data-ingestion-spark/workflow_de_ml.png" width=70%>
</center>


- As a data engineer, you need a general idea of how machine learning works—there is no need to worry about the math behind it.
- It's more important to focus on the engineering side of data science since we might need to team up with ML engineers for deployment and infrastructure.
- So, it’s helpful to have a bit of Machine Learning knowledge and it can give good growth opportunities for Data Engineer

### Machine Learning Workflow
---

- As a Data Engineer, we also must know about Machine Learning Workflow
- In general, ML Workflow is similar to Data Pipeline Workflow
- We will learn the simplified version of ML Workflow

<center>
    <img src="https://sekolahdata-assets.s3.ap-southeast-1.amazonaws.com/notebook-images/mde-data-ingestion-spark/ml_workflow.png" width=50%>
</center>

## Introduction to Machine Learning Pipeline
---

### Data Preparation
---

- We already learned that ML models need data to understand the patterns and make a prediction
- The workflow is similar to Data engineering when creating a Data Pipeline, the first step is preparing the data
- But, the difference is usually the data ingested by the Data Scientist is already in the correct format and it's in one place like a Data Warehouse or File System
- In this step, after Data Scientist ingest the data they were going to explore the data to see:
    - The data distribution
    - Missing values
    - Duplicate data
    - etc

- Data Preparation steps that they are going to do are:
    - Create a connection to Database Warehouse
    - Cleaning Data / Data Preprocessing
    - Splitting data to Train and Test

---


# <font color='blue'>1. Study Case 1: Bank Marketing
---

**Background:**
- In a Bank Company, there is a Bank Marketing data source in [Google Spreadsheet](https://docs.google.com/spreadsheets/d/1d-xwYjTVby284BZxKCt9XuxOY_x11C7pXSa7xcXpKLw/edit?usp=sharing)
- That data contains the last marketing campaign the bank performed
- Data Scientists want to use that data to find the best strategies to improve for the next marketing campaign!
- Data Scientists also want to build a Machine Learning model to identify the patterns that later can help them to find conclusions to develop better future strategies
- But there is a problem:
    - The data is not in the correct format
    - The data is also dirty, and not compatible with building Machine Learning model
    - Data Scientist who will build a Machine Learning model will struggle because the data is in Google Spreadsheet

**Solution:**
- The Data Engineer will build a Data Pipeline to move the data from Google Spreadsheet to the Data Warehouse
- When building a Data Pipeline, there are two layers:
    - Staging Layer
    - Warehouse Layer
- For database services, you can use this [Docker Compose](https://github.com/Kurikulum-Sekolah-Pacmann/machine_learning_pipeline/blob/main/docker-compose.yml)
- In Data Pipeline, we will do a simple Transformation
- After the data in the Data Warehouse, we will also build a Machine Learning Pipeline
- In the Machine Learning pipeline, we will get the data from the Data Warehouse
- After that, we will do Preprocessing tasks like changing data type, mapping values, splitting data, etc
- Then, in the last step we will create a Machine Learning Model using Decision Tree
- All of the processes or components above, we will store the information process in a Log File

**Proposed Workflow**

<center>
    <img src="https://sekolahdata-assets.s3.ap-southeast-1.amazonaws.com/notebook-images/mde-data-ingestion-spark/workflow_liveclass_w8.png" width=100%>
</center>

The components for this case are:
- Extract data from Spreadsheet
- Dump raw data to Staging Area
- Extract from Staging then Transform
- Dump clean data to Warehouse Area
- Extract data from Warehouse
- Preprocessing based on the requirements
- Create Machine Learning model

Also we will create all the components into script based using this directory as a reference

<center>
    <img src="https://sekolahdata-assets.s3.ap-southeast-1.amazonaws.com/notebook-images/mde-data-ingestion-spark/directory_script.png">
</center>

In [26]:
import pandas as pd

### **0. Setup Script Configuration**

- The first step to implement our solution, is to create a configuration function for:
    - Connect to Spreadsheet --> `auth_gspread()`
    - Connect to Staging and Warehouse Database --> `init_engine()`
    - Create Logging Process --> `log_process()`

- All of the function is stored in `src/utils/helper.py`

In [1]:
from src.utils.helper import auth_gspread, init_engine, logging_process

### **1. Extract from Spreadsheet**
---

- The next step is to create a function called `extract_spreadsheet_process()` to read the data from Spreadsheet
- In the function, we can also use `auth_gspread()` function
- Make sure you're already create a copy of the data source Spreadsheet and already have the JSON credentials from Google Cloud Platform
- Save the script in the directory `src/staging/extract/extract_spreadsheet.py`

In [2]:
from src.staging.extract.extract_spreadsheet import extract_spreadsheet_process

In [3]:
df_bank = extract_spreadsheet_process(worksheet_name = "bank_marketing")

df_bank.head()

===== Start Extracting Spreadsheet data =====


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
1,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
2,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
3,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
4,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
5,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


### **2. Dump data to Staging Layer**
---

- For easier use, we will dump the Spreadsheet data into a Staging Layer
- To dump the data into the Database, we can use a helper function called `init_engine()` to connect to the database
- Then, we create a function called `load_to_staging()` to dump all the data into Staging Layer
- Save the script into the directory `src/staging/load/load_data.py`

In [4]:
from src.staging.load.load_data import load_to_staging

Now, dump the spreadsheet data into Staging Database on `stg_bank_marketing` table

In [5]:
load_to_staging(data = df_bank,
                table_name = "stg_bank_marketing")

===== Start Load to Staging Database =====
===== Finish Load to Staging Database =====


### **3. Extract Data from Staging and Transform**
---

- Create a function called `extract_staging_process()` in the directory `stg/warehouse/extract/extract_staging.py`
- The process is simple, because it's just read the data from Staging Layer

In [7]:
from src.warehouse.extract.extract_staging import extract_staging_process

Read the data in the `stg_bank_marketing` table

In [8]:
df_new_data = extract_staging_process(table_name = "stg_bank_marketing")

df_new_data.head()

===== Start Extracting data from Staging =====
===== Finish Extracting data from Staging =====


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


#### Transform Data

- After we explore the data, turns out there's some data that we need to transform:
    - Remove dot `.` from the `job` column
    - Filter the data from `balance` when the values are less than zero
    - Change data types in columns `[age, balance, day, campaign, pdays, previous, duration]` into numeric data

- We will create a function for each component!
- So we will need three functions:
    - `remove_dot_process()` in the directory `src/warehouse/transform/process/remove_dot.py`
    - `filter_balance_process()` in the directory `src/warehouse/transform/process/filter_balance.py`
    - `casting_data_process()` in the directory `src/warehouse/transform/process/casting_data.py`

- After that, we will call all the functions in the main transform script in the directory `src/warehouse/transform/transform_data.py`
- Store all the functions that are called into `transform_process()`

remove dot `.` in `job`

In [9]:
df_new_data["job"] = df_new_data["job"].str.replace(".", "", regex = False)

df_new_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


filter `balance` less than zero

In [10]:
df_new_data = df_new_data[df_new_data["balance"].astype(int) >= 0]

df_new_data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [11]:
# validate the results
df_new_data[df_new_data["balance"].astype(int) < 0]

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit


- change data type `age, balance, day, campaign, pdays, previous, duration` convert to numeric

In [12]:
# dictionary for casting
COLS_TO_CAST = {
    "age": "int",
    "balance": "float",
    "day": "int",
    "campaign": "int",
    "pdays": "int",
    "previous": "int",
    "duration": "int"
}

df_new_data = df_new_data.astype(COLS_TO_CAST)

df_new_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10474 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        10474 non-null  int64  
 1   job        10474 non-null  object 
 2   marital    10474 non-null  object 
 3   education  10474 non-null  object 
 4   default    10474 non-null  object 
 5   balance    10474 non-null  float64
 6   housing    10474 non-null  object 
 7   loan       10474 non-null  object 
 8   contact    10474 non-null  object 
 9   day        10474 non-null  int64  
 10  month      10474 non-null  object 
 11  duration   10474 non-null  object 
 12  campaign   10474 non-null  int64  
 13  pdays      10474 non-null  int64  
 14  previous   10474 non-null  int64  
 15  poutcome   10474 non-null  object 
 16  deposit    10474 non-null  object 
dtypes: float64(1), int64(5), object(11)
memory usage: 1.4+ MB


Call the main transform function

In [13]:
from src.warehouse.transform.transform_data import transform_process

In [14]:
df_new_data = transform_process(data = df_new_data)

df_new_data.head()

===== Start Transform Data ===== 

===== Finish Transform Data =====


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin,married,secondary,no,2343.0,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin,married,secondary,no,45.0,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270.0,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476.0,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin,married,tertiary,no,184.0,no,no,unknown,5,may,673,2,-1,0,unknown,yes


### **4. Dump the Clean Data to Warehouse**
---

- For easier use, we will dump the Spreadsheet data into a Staging Layer
- To dump the data into the Database, we can use a helper function called `init_engine()` to connect to the database
- Then, we create a function called `load_to_warehouse()` to dump all the data into Staging Layer
- Save the script into the directory `src/staging/load/load_data.py`

In [15]:
from src.warehouse.load.load_data import load_to_warehouse

We will dump the table into `wh_bank_marketing` table

In [16]:
load_to_warehouse(data = df_new_data,
                  table_name = "wh_bank_marketing")

===== Start Load to Warehouse Database =====
===== Finish Load to Warehouse Database =====


### **5. Extract data from Warehouse**
---

- Now we're in the Machine Learning Pipeline
- In this Pipeline we will:
    - Extract the data from Data Warehouse
    - Preprocessing
    - Splitting Data
    - Modeling

- Now, we will create a function called `extract_warehouse_process()` in the directory `src/modeling/extract/extract_warehouse.py`
- Just like another extract process, this process only extracting data from Database
- So in the script, we will use `init_engine()` function to connect to the Data Warehouse

In [1]:
from src.modeling.extract.extract_warehouse import extract_warehouse_process

We will read the bank marketing data in the Warehouse on the `wh_bank_marketing` table

In [2]:
df_wh_bank = extract_warehouse_process(table_name = "wh_bank_marketing")

df_wh_bank.head()

===== Start Extracting data from Warehouse =====
===== Finish Extracting data from Warehouse =====


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin,married,secondary,no,2343.0,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin,married,secondary,no,45.0,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270.0,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476.0,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin,married,tertiary,no,184.0,no,no,unknown,5,may,673,2,-1,0,unknown,yes


### **6. Preprocessing**
---

- For this step, we will determine the column that will be used as a Target for Machine Learning Model
- Turns out in our Bank Marketing data, the column that will be used as a Target is the `deposit` column
- Before that, we will do a Preprocessing first to make sure our data is ready for use for our Machine Learning Model
- The preprocessing tasks in this case are:
    - Mapping values in the columns `[default, housing, loan, deposit]` into `0` for `no` and `1` for `yes`
    - Create One Hot Encoding for columns `[job, marital, education, contact, poutcome]`. Because the ML Model, can't take a string of categorical data. They can only take numeric data
    - Mapping values in the `month` values to a numeric

- Just like in Transformation process, we will create a function for each process Preprocessing
    - Mapping Values `0` and `1` --> `mapping_values_process()` in the directory `src/modeling/preprocessing/process/mapping_values.py`
    - One Hot Encoding --> `ohe_process()` in the directory `src/modeling/preprocessing/process/one_hot_encoding.py`
    - Mapping Month --> `mapping_month_process()` in the directory `src/modeling/preprocessing/process/mapping_month.py` 

- After that, we will run all the function in the main preprocessing transform script in the directory `src/modeling/preprocessing/preprocessing_data.py`
- Store all the functions on the `preprocessing_process()`

Mapping values to `0` and `1`

In [23]:
# before mapping

COLS_TO_MAP = [
    "default",
    "housing",
    "loan",
    "deposit"
]

for col in COLS_TO_MAP:
    print(f"Column {col} have a values: {df_wh_bank[col].unique()}")

Column default have a values: ['no' 'yes']
Column housing have a values: ['yes' 'no']
Column loan have a values: ['no' 'yes']
Column deposit have a values: ['yes' 'no']


In [24]:
VALUES_TO_MAP = {"yes": 1, "no": 0}

print("===== Start Mapping Values =====")

for col in COLS_TO_MAP:
    df_wh_bank[col] = df_wh_bank[col].map(VALUES_TO_MAP)

print("===== Finish Mapping Values =====")

===== Start Mapping Values =====
===== Finish Mapping Values =====


One Hot Encoding Process

In [ ]:
CHECK_COLS = [
    "job",
    "marital",
    "education",
    "contact",
    "poutcome",
]

df_wh_bank = pd.get_dummies(df_wh_bank, columns = CHECK_COLS)

df_wh_bank.head()

Mapping Month values

In [ ]:
DICT_MONTH = {
    "jan": 1,
    "feb": 2,
    "mar": 3,
    "apr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "aug": 8,
    "sep": 9,
    "oct": 10,
    "nov": 11,
    "dec": 12
}

df_wh_bank["month"] = df_wh_bank["month"].map(DICT_MONTH)

Run the preprocessing main function

In [3]:
from src.modeling.preprocessing.preprocessing_data import preprocessing_process

In [4]:
df_wh_bank = preprocessing_process(data = df_wh_bank)

df_wh_bank.head()

===== Start Preprocessing Data =====
===== Start Mapping Values =====
===== Finish Mapping Values =====
===== Start One Hot Encoding Process 

===== Finished One Hot Encoding Process =====
===== Start Mapping Month ===== 

===== Finish Mapping Month =====
===== Failed Preprocessing Data =====


,age,default,balance,housing,loan,day,month,duration,campaign,pdays,...,education_secondary,education_tertiary,education_unknown,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,59,0,2343.0,1,0,5,5,1042,1,-1,...,True,False,False,False,False,True,False,False,False,True
1,56,0,45.0,0,0,5,5,1467,1,-1,...,True,False,False,False,False,True,False,False,False,True
2,41,0,1270.0,1,0,5,5,1389,1,-1,...,True,False,False,False,False,True,False,False,False,True
3,55,0,2476.0,1,0,5,5,579,1,-1,...,True,False,False,False,False,True,False,False,False,True
4,54,0,184.0,0,0,5,5,673,2,-1,...,False,True,False,False,False,True,False,False,False,True


### **7. Splitting Data**
---

- After we do a preprocessing of our data, there's one step that we must do before creating an ML Model
- The process is called Splitting Data
- We will create a function called `splitting_process()` in the directory `src/modeling/preprocessing/splitting_data.py`

In [5]:
from src.modeling.preprocessing.splitting_data import splitting_process

Now we will split the data using test size 20% and set `deposit` column as a target data

In [7]:
X_train, X_test, y_train, y_test = splitting_process(data = df_wh_bank,
                                                     target_col = "deposit",
                                                     test_size = 0.2)

===== Start Splitting Data =====

Features Shape: (10474, 37)
Target Shape: (10474,)
Train Features Shape: (8379, 37)
Test Features Shape: (2095, 37)
Train Target Shape: (8379,)
Test Target Shape: (2095,)

===== Finished splitting data =====


### **8. Modeling**
---

- The last step of this case is to create an ML Model
- In this case, we will build an ML Model using a Decision Tree
- Create the ML Model in the directory `src/modeling/decision_tree.py` on the function `modeling_process()`
- In the function after do a modeling, print the result for training and test

In [8]:
from src.modeling.decision_tree import modeling_process

In [9]:
modeling_process(X_train = X_train,
                 X_test = X_test,
                 y_train = y_train,
                 y_test = y_test)

===== Start Modeling Data =====

Decision Tree training accuracy 1.0
Decision Tree test accuracy 0.7770883054892601
===== Finish Modeling Data =====


### **9. Modeling**
---

For the last step, we will compile all the process in the `pipeline.py` script!

You can access the code on this repository https://github.com/Kurikulum-Sekolah-Pacmann/machine_learning_pipeline